# Simulating Kafka Resiliency with Pod Topology Constraints in Kubernetes
----------

If you would like to run this notebook live, you can create a codespace using the devcontainer in this repository. The devcontainer has all the necessary tools and dependencies installed. Once launched, you can open this notebook, click Clear All Outputs and run the cells. 

## Introduction
In this notebook, we will simulate a Kafka cluster with 3 brokers in a k3s environment. We will use Pod Topology Constraints to ensure that the Kafka brokers and Zookeeper nodes are scheduled on different nodes. We will then simulate a node failure and observe how the Kafka cluster behaves.

## Step 1: Create a k3s cluster
Note: We will label the k3s nodes with topology.kubernetes.io/zone so that we can use Pod Topology Constraints to ensure that the Kafka brokers and Zookeeper nodes are scheduled on different nodes.

In [1]:
!/usr/local/bin/k3d cluster create kube-cluster \
  --agents 3 \
  --k3s-node-label topology.kubernetes.io/zone=zone-a@agent:0 \
  --k3s-node-label topology.kubernetes.io/zone=zone-b@agent:1 \
  --k3s-node-label topology.kubernetes.io/zone=zone-c@agent:2

INFO[0000] Prep: Network                                
INFO[0000] Created network 'k3d-kube-cluster'           
INFO[0000] Created image volume k3d-kube-cluster-images 
INFO[0000] Starting new tools node...                   
INFO[0000] Pulling image 'ghcr.io/k3d-io/k3d-tools:5.6.0' 
INFO[0001] Creating node 'k3d-kube-cluster-server-0'    
INFO[0002] Pulling image 'docker.io/rancher/k3s:v1.27.4-k3s1' 
INFO[0002] Starting Node 'k3d-kube-cluster-tools'       
INFO[0005] Creating node 'k3d-kube-cluster-agent-0'     
INFO[0006] Creating node 'k3d-kube-cluster-agent-1'     
INFO[0006] Creating node 'k3d-kube-cluster-agent-2'     
INFO[0006] Creating LoadBalancer 'k3d-kube-cluster-serverlb' 
INFO[0007] Pulling image 'ghcr.io/k3d-io/k3d-proxy:5.6.0' 
INFO[0010] Using the k3d-tools node to gather environment information 
INFO[0010] HostIP: using network gateway 172.18.0.1 address 
INFO[0010] Starting cluster 'kube-cluster'              
INFO[0010] Starting servers...                         

## Step 2: Deploy Kafka
The following stateful set can be found under deploy/kafka/01-sts.yaml. It deploys a Kafka cluster with 3 brokers. The stateful set uses Pod Topology Constraints to ensure that the Kafka brokers are scheduled on different nodes.

We will also deploy a headless service for the Kafka brokers.

In [12]:
!kubectl apply -f deploy/kafka/01-sts.yaml

serviceaccount/kafka unchanged
service/kafka-headless unchanged
statefulset.apps/kafka created


Notice that each pod is running on a different node.

In [87]:
! kubectl get pods -n kafka -o wide

NAME      READY   STATUS    RESTARTS   AGE     IP          NODE                       NOMINATED NODE   READINESS GATES
kafka-0   1/1     Running   0          3m37s   10.42.0.4   k3d-kube-cluster-agent-0   <none>           <none>
kafka-1   1/1     Running   0          3m37s   10.42.2.5   k3d-kube-cluster-agent-2   <none>           <none>
kafka-2   1/1     Running   0          3m37s   10.42.1.4   k3d-kube-cluster-agent-1   <none>           <none>


With the stateful set and headless service deployed, we can now test the Kafka cluster.
We will exec into one of the Kafka brokers and create a topic called "test".

In [85]:
! kubectl -n kafka exec -it kafka-0 -- kafka-topics --create --topic test --partitions 3 --replication-factor 3 --bootstrap-server kafka-0.kafka-headless.kafka.svc.cluster.local:9092

Created topic test.


With the topic created, if we run a describe topic command, we should see the topic "test" with 3 partitions and a replication factor of 3.



In [86]:
! kubectl -n kafka exec -it kafka-0 -- kafka-topics --describe --topic test --bootstrap-server kafka-0.kafka-headless.kafka.svc.cluster.local:9092

Topic: test     TopicId: WmMXgsr2RcyZU9ohfoTUWQ PartitionCount: 3       ReplicationFactor: 3    Configs: 
        Topic: test     Partition: 0    Leader: 0       Replicas: 0,1,2 Isr: 0,1,2
        Topic: test     Partition: 1    Leader: 1       Replicas: 1,2,0 Isr: 1,2,0
        Topic: test     Partition: 2    Leader: 2       Replicas: 2,0,1 Isr: 2,0,1


## Step 3: Simulate a node failure
We will simulate a node failure by scaling the stateful set down to 2 replicas. We will then describe the topic again and observe the change in the number of in-sync replicas.

In [ ]:
! kubectl -n kafka scale sts kafka --replicas=2

In [ ]:
! kubectl -n kafka exec -it kafka-0 -- kafka-topics --describe --topic test --bootstrap-server kafka-0.kafka-headless.kafka.svc.cluster.local:9092